In [61]:
from llama_index.core.base.llms.types import ChatMessage, MessageRole
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    select,
)
from sqlmodel import Session

In [62]:
engine = create_engine("sqlite:///:memory:")
metadata_obj = MetaData()

In [63]:
sqlite_file_name = "database.db"
sqlite_url = f"sqlite:///{sqlite_file_name}"
connect_args = {"check_same_thread": False}
engine = create_engine(sqlite_url, connect_args=connect_args)

In [64]:
# create city SQL table
table_name = "city_stats"
city_stats_table = Table(
    table_name,
    metadata_obj,
    Column("city_name", String(16), primary_key=True),
    Column("population", Integer),
    Column("country", String(16), nullable=False),
)
metadata_obj.create_all(engine)

In [65]:
from llama_index.core import SQLDatabase
sql_database = SQLDatabase(engine, include_tables=["city_stats"])

from sqlalchemy import insert

rows = [
    {"city_name": "Toronto", "population": 2930000, "country": "Canada"},
    {"city_name": "Tokyo", "population": 13960000, "country": "Japan"},
    {
        "city_name": "Chicago",
        "population": 2679000,
        "country": "United States",
    },
    {"city_name": "Seoul", "population": 9776000, "country": "South Korea"},
]
for row in rows:
    stmt = insert(city_stats_table).values(**row)
    with engine.begin() as connection:
        cursor = connection.execute(stmt)

IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: city_stats.city_name
[SQL: INSERT INTO city_stats (city_name, population, country) VALUES (?, ?, ?)]
[parameters: ('Toronto', 2930000, 'Canada')]
(Background on this error at: https://sqlalche.me/e/20/gkpj)

In [66]:
# view current table
stmt = select(
    city_stats_table.c.city_name,
    city_stats_table.c.population,
    city_stats_table.c.country,
).select_from(city_stats_table)

with engine.connect() as connection:
    results = connection.execute(stmt).fetchall()
    print(results)

[('Toronto', 2930000, 'Canada'), ('Tokyo', 13960000, 'Japan'), ('Chicago', 2679000, 'United States'), ('Seoul', 9776000, 'South Korea')]


In [67]:
from sqlalchemy import text

with engine.connect() as con:
    rows = con.execute(text("SELECT city_name from city_stats"))
    for row in rows:
        print(row)

('Chicago',)
('Seoul',)
('Tokyo',)
('Toronto',)


In [84]:
from llama_index.llms.ollama import Ollama
from llama_index.core.settings import Settings
from llama_index.embeddings.ollama import OllamaEmbedding

llm = Ollama(model="llama3.1", request_timeout=420)
embed_model = OllamaEmbedding(model_name="nomic-embed-text")
Settings.llm = llm
Settings.embed_model = embed_model

In [85]:
from llama_index.core.query_engine import NLSQLTableQueryEngine

query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database, tables=["city_stats"], llm=llm
)
query_str = "Which city has the highest population?"
response = query_engine.query(query_str)

In [86]:
print(response)

Based on the query results, the response would be:

"The city with the highest population is Tokyo, with a population of approximately 13.96 million."


In [87]:
from llama_index.core.indices.struct_store.sql_query import (
    SQLTableRetrieverQueryEngine,
)
from llama_index.core.objects import (
    SQLTableNodeMapping,
    ObjectIndex,
    SQLTableSchema,
)
from llama_index.core import VectorStoreIndex

# set Logging to DEBUG for more detailed outputs
table_node_mapping = SQLTableNodeMapping(sql_database)
table_schema_objs = [
    (SQLTableSchema(table_name="city_stats"))
]  # add a SQLTableSchema for each table

obj_index = ObjectIndex.from_objects(
    table_schema_objs,
    table_node_mapping,
    VectorStoreIndex,
)
query_engine = SQLTableRetrieverQueryEngine(
    sql_database, obj_index.as_retriever(similarity_top_k=1)
)

In [88]:
response = query_engine.query("Which city has the highest population?")
print(response)

The city with the highest population is Tokyo.


In [97]:
from llama_index.core.agent import ReActAgent
from llama_index.core.tools import QueryEngineTool, ToolMetadata, FunctionTool
from llama_index.core.base.llms.types import ChatMessage, MessageRole

def query_through_sql(query: str):
    """Queries """
    return query_engine.query(query)

tools = [
    QueryEngineTool.from_defaults(
        query_engine=query_engine,
        name="sql_tool",
        description="Query engine for querying through DB table 'city_stats' with SQL.",
        # TODO: mit Pandas, SQL tool, liste alle Spalten der Tabelle in descrption
    )
]

chat_history = [
    ChatMessage(
        role=MessageRole.SYSTEM,
        content="""
        You are Anna Pham responsible for HR duties.
        Your role is to assist with a variety of tasks, including answering general questions, providing summaries, and performing HR-related analyses.
        ## Language
        - You speak English, Vietname  and German
        - You answer in German mostly. Only speak the language you can talk with.

        ## Conversation Style
        - You engage in natural conversations and answer simple questions directly, without using tools.
        - When explicitly asked to use a tool (e.g., "Use the tool for..."), you follow the request accordingly.
        - For HR-related queries or document-related tasks, you utilize the appropriate tools to provide structured responses.
        - When the user requests for a listing, show the thoughts you process from a tool to the user.
        - You communicate with the user in Markdown language, for easier formatting in a Frontend application.

        ## Tools
        You have access to several tools that help accomplish tasks effectively.
        You should determine when and how to use them to complete requests efficiently.
        If a task requires multiple steps, you can break it down and apply different tools as needed.
        Available tools:
        {tool_desc}

        ## Output Format
        When using a tool, follow this structured format:
        Thought: I need to use a tool to complete this request. Action: [Tool name] (one of {tool_names})
        Action Input: [Valid JSON format input]

        Always start with a Thought before taking action.

        If a tool is used, the system will respond in the following format:
        Observation: [Tool response]
        You should continue this process until you have gathered enough information to respond to the query.
        Once you have enough details, conclude with one of the following:

        Thought: I have sufficient information to answer.
        Answer: [Your answer]

        OR

        Thought: The available tools do not provide the necessary information.
        Answer: Sorry, I cannot answer this query.
        The output must be formatted in Markdown with the thoughts!

        ## Additional Rules
        - When answering a direct question (e.g., "What is your name?"), respond naturally without invoking tools.
        - Always follow the expected function signature of each tool and provide the necessary arguments.
        - Use bullet points to explain the reasoning behind complex responses, especially when using tools.
        - If the user explicitly requests tool usage (e.g., "Use the HR tool for..."), follow the instruction exactly.

        ## Current Conversation
        Below is the conversation history, which you should consider when providing responses:
        [Include conversation history here]
        """
    )
]

agent = ReActAgent.from_tools(
    tools=tools,
    llm=llm,
    verbose=True,
    chat_history=chat_history,
)

In [98]:
response = agent.chat("Use sql_tool. List down all columns inside the table city_stats.")
print(response)

> Running step f3a75797-907c-464e-b24f-d2f3cfc1352d. Step input: Use sql_tool. List down all columns inside the table city_stats.
Thought: I need to use a tool to list down all columns inside the table 'city_stats'.
Action: sql_tool
Action Input: {'input': "SELECT column_name FROM information_schema.columns WHERE table_name = 'city_stats'"}
Observation: It seems like there's been a misunderstanding here.

The SQL query `SELECT * FROM city_stats LIMIT 0` doesn't actually return any data because of the `LIMIT 0` clause. It's not an error, it's intended behavior in this context. The explanation provided is trying to justify why this query works as expected, but it contains an incorrect assumption: that the query itself can be executed and return the column names without being part of a larger SQL statement.

The response you're looking for should indicate that the query is valid in its intention, even though executing it directly does not produce any output. Here's how I'd adjust the resp

In [95]:
response = agent.chat("What tools do you provide at the moment?")
print(response)

> Running step 4429399a-87bf-46d4-97d2-d6be4f118f40. Step input: What tools do you provide at the moment?
Thought: I need to list out the available tools.
Action: tool
Action Input: {'properties': AttributedDict([('input', AttributedDict([('title', 'Input'), ('type', 'string')]))]), 'required': ['input'], 'type': 'object'}
Observation: Error: No such tool named `tool`.
> Running step 272f73c5-e3a9-4821-914c-8a25475343e5. Step input: None
Thought: It seems that the tool "tool" is not available.
Action: sql_tool
Action Input: {'input': 'Available tools are: sql_tool'}
Observation: It seems like you're trying to synthesize a response from the query results, but there's no actual query result. The original input was not a valid question with a clear intent to retrieve specific data from a database.

However, based on your input and the SQL provided in the response, it appears that the user is asking about available tools but doesn't provide enough context for a meaningful query. To synthes

In [96]:
response = agent.chat("Use sql_tool. Which city has the highest population?")
print(response)

> Running step 187601b7-d046-4e07-8784-193bae833ceb. Step input: Use sql_tool. Which city has the highest population?
Thought: I need to use a tool to find out which city has the highest population.
Action: sql_tool
Action Input: {'input': 'SELECT city_name FROM city_stats ORDER BY population DESC LIMIT 1'}
Observation: The most populous city is Tokyo.
> Running step 9af4188c-de60-4b22-9dfd-382e0ee7cfd6. Step input: None
Thought: Ich verstehe, dass die Observation bestätigt, dass die größte Stadt in der Datenbank tatsächlich "Tokyo" ist.
Answer: Die größte Stadt laut unserer Datenbank ist Tokyo.
Die größte Stadt laut unserer Datenbank ist Tokyo.


In [67]:
!pip install sqlparse psycopg2-binary psycopg2

  Using cached psycopg2-2.9.10.tar.gz (385 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for psycopg2 (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [48 lines of output]
      /tmp/pip-build-env-pooi2_at/overlay/lib/python3.12/site-packages/setuptools/dist.py:751: SetuptoolsDeprecationWarning: License classifiers are deprecated.
      !!
      
              ********************************************************************************
              Please consider removing the following classifiers in favor of a SPDX license expression:
      
              License :: OSI Approved :: GNU Library or Lesser General Public License (LGPL)
      
              See https://packaging.python.org/en/latest/guides/writing-pyproject-toml/#license for details.
              ****************************************************

In [85]:
import os
import re
import subprocess
import uuid
import psycopg2
from psycopg2 import sql

def import_any_sql_to_postgres(dump_file,
                              pg_host='localhost',
                              pg_port='5432',
                              pg_user='postgres',
                              pg_password='',
                              target_db=None):
    """
    Import SQL dump from any source (MySQL/MariaDB/PostgreSQL) into PostgreSQL database.

    Parameters:
    - dump_file (str): Path to the SQL dump file
    - pg_host (str): PostgreSQL host
    - pg_port (str): PostgreSQL port
    - pg_user (str): PostgreSQL username
    - pg_password (str): PostgreSQL password
    - target_db (str): Target database name (will be auto-generated if None)

    Returns:
    - str: Name of the created database
    """
    # Check if dump file exists
    if not os.path.isfile(dump_file):
        raise FileNotFoundError(f"SQL dump file not found: {dump_file}")

    # Generate database name if not provided
    if not target_db:
        target_db = f"imported_db_{uuid.uuid4().hex[:6]}"

    # Setup database parameters
    db_params = {
        'host': pg_host,
        'port': pg_port,
        'user': pg_user,
        'password': pg_password
    }

    # Create new database
    create_database(db_params, target_db)

    # Identify dump type
    dump_type = identify_dump_type(dump_file)

    # Import based on dump type
    if dump_type == 'postgres':
        success = import_postgres_dump(dump_file, db_params, target_db)
    else:
        success = import_mysql_dump(dump_file, db_params, target_db)

    if success:
        print(f"Successfully imported {dump_type} dump into PostgreSQL database '{target_db}'")
    else:
        print(f"Failed to import {dump_type} dump into '{target_db}'")

    return target_db

def identify_dump_type(dump_file):
    """Identify if the dump is from MySQL/MariaDB or PostgreSQL"""
    try:
        with open(dump_file, 'r', encoding='utf-8', errors='ignore') as f:
            content = f.read(10000)  # Read first 10000 chars to identify

            # PostgreSQL specific markers
            if re.search(r'SET statement_timeout|SET lock_timeout|SET client_encoding|SET standard_conforming_strings', content):
                return 'postgres'

            # MySQL/MariaDB specific markers
            if re.search(r'-- MySQL dump|-- MariaDB dump|ENGINE=|AUTO_INCREMENT=|DEFAULT CHARSET=', content):
                return 'mysql'

            # Additional check - MySQL typically uses backticks for identifiers
            if '`' in content:
                return 'mysql'

            # Default to postgres if we can't determine
            return 'postgres'
    except Exception:
        # If there's any issue reading or parsing the file
        # Default to postgres
        return 'postgres'

def create_database(db_params, db_name):
    """Create a new database in PostgreSQL"""
    # Connect to default database
    conn_params = db_params.copy()
    conn_params['dbname'] = 'postgres'  # Connect to default postgres db

    conn = psycopg2.connect(**conn_params)
    conn.autocommit = True
    cursor = conn.cursor()

    # Check if database exists
    cursor.execute(sql.SQL("SELECT 1 FROM pg_database WHERE datname = %s"), (db_name,))
    if not cursor.fetchone():
        cursor.execute(sql.SQL("CREATE DATABASE {}").format(sql.Identifier(db_name)))
        print(f"Created database '{db_name}'")
    else:
        print(f"Database '{db_name}' already exists, using it")

    cursor.close()
    conn.close()

def convert_mysql_to_postgres(content):
    """Convert MySQL/MariaDB SQL syntax to PostgreSQL compatible syntax"""
    # Replace backticks with double quotes for identifiers
    content = re.sub(r'`([^`]*)`', r'"\1"', content)

    # Remove or replace MySQL/MariaDB specific syntax
    content = re.sub(r'ENGINE\s*=\s*\w+', '', content)
    content = re.sub(r'AUTO_INCREMENT\s*=\s*\d+', '', content)
    content = re.sub(r'DEFAULT CHARSET\s*=\s*\w+', '', content)
    content = re.sub(r'COLLATE\s*=\s*\w+', '', content)
    content = re.sub(r'CHARACTER SET\s*\w+', '', content)

    # Data types conversions
    content = re.sub(r'int\(\d+\)', 'integer', content)
    content = re.sub(r'tinyint\(\d+\)', 'smallint', content)
    content = re.sub(r'mediumint\(\d+\)', 'integer', content)
    content = re.sub(r'bigint\(\d+\)', 'bigint', content)
    content = re.sub(r'varchar\((\d+)\)', r'varchar(\1)', content)
    content = re.sub(r'UNSIGNED', '', content)

    # Handle AUTO_INCREMENT
    content = re.sub(r'AUTO_INCREMENT', 'SERIAL', content)

    # Handle BOOLEAN values
    content = re.sub(r"b'0'", 'false', content)
    content = re.sub(r"b'1'", 'true', content)

    # Additional common MySql functions
    content = re.sub(r'IFNULL\(', 'COALESCE(', content)

    # Remove MySQL comment delimiters
    content = re.sub(r'/\*!.*?\*/', '', content)

    return content

def import_mysql_dump(dump_file, db_params, db_name):
    """Import MySQL/MariaDB dump to PostgreSQL without pgloader"""
    print(f"Importing MySQL/MariaDB dump to PostgreSQL: {dump_file}")

    try:
        with open(dump_file, 'r', encoding='utf-8', errors='ignore') as f:
            sql_content = f.read()

        # Convert MySQL syntax to PostgreSQL
        sql_content = convert_mysql_to_postgres(sql_content)

        # Split into executable statements
        statements = re.split(r';[ \t]*\n', sql_content)

        # Connect to the target database
        conn_params = db_params.copy()
        conn_params['dbname'] = db_name
        conn = psycopg2.connect(**conn_params)
        conn.autocommit = False
        cursor = conn.cursor()

        # Execute each statement
        success_count = 0
        total_statements = len(statements)
        error_count = 0

        for i, statement in enumerate(statements):
            if statement.strip():
                try:
                    cursor.execute(statement)
                    success_count += 1

                    # Commit every 100 statements or when reaching the final one
                    if success_count % 100 == 0 or i == total_statements - 1:
                        conn.commit()
                        print(f"Progress: {success_count}/{total_statements} statements processed")

                except Exception as e:
                    error_count += 1
                    print(f"Error executing statement: {str(e)[:100]}...")
                    conn.rollback()  # Rollback the failed statement

        cursor.close()
        conn.close()

        print(f"Import completed: {success_count} statements succeeded, {error_count} failed")
        return error_count < total_statements * 0.5  # Consider successful if less than 50% failed
    except Exception as e:
        print(f"Import failed with error: {e}")
        return False

def import_postgres_dump(dump_file, db_params, db_name):
    """Import PostgreSQL dump to PostgreSQL"""
    print(f"Importing PostgreSQL dump: {dump_file}")

    # Set environment variable for password
    env = os.environ.copy()
    env['PGPASSWORD'] = db_params['password']

    # Import using psql or pg_restore based on file type
    try:
        if dump_file.endswith('.dump') or dump_file.endswith('.backup'):
            # Binary dump
            cmd = [
                'pg_restore',
                '--no-owner',
                '--no-privileges',
                f"--host={db_params['host']}",
                f"--port={db_params['port']}",
                f"--username={db_params['user']}",
                f"--dbname={db_name}",
                dump_file
            ]
        else:
            # Text dump (SQL file)
            cmd = [
                'psql',
                f"--host={db_params['host']}",
                f"--port={db_params['port']}",
                f"--username={db_params['user']}",
                f"--dbname={db_name}",
                '--file', dump_file
            ]

        result = subprocess.run(cmd, env=env, capture_output=True, text=True)

        # Print any output from the command
        if result.stdout:
            print("Output:", result.stdout[:500], "..." if len(result.stdout) > 500 else "")
        if result.stderr:
            print("Errors/Warnings:", result.stderr[:500], "..." if len(result.stderr) > 500 else "")

        return True
    except Exception as e:
        print(f"Import using command-line tools failed: {e}")

        # Try direct SQL execution as fallback
        try:
            print("Trying direct SQL execution as fallback...")
            conn_params = db_params.copy()
            conn_params['dbname'] = db_name
            conn = psycopg2.connect(**conn_params)
            conn.autocommit = True
            cursor = conn.cursor()

            with open(dump_file, 'r', encoding='utf-8', errors='ignore') as f:
                sql_content = f.read()

            cursor.execute(sql_content)
            cursor.close()
            conn.close()
            return True
        except Exception as e2:
            print(f"Direct SQL execution also failed: {e2}")
            return False

# Example usage in Jupyter:
db_name = import_any_sql_to_postgres('./data/customer.sql',
                                    pg_host='localhost',
                                    pg_user='postgres',
                                    pg_password='password',
                                     target_db="unique_db")

Database 'unique_db' already exists, using it
Importing PostgreSQL dump: ./data/customer.sql
Errors/Warnings: psql:data/customer.sql:16: ERROR:  relation "customer" already exists
psql:data/customer.sql:18: ERROR:  null value in column "id" of relation "customer" violates not-null constraint
DETAIL:  Failing row contains (null, Emilia Dunsford, Scott, edunsford0@canalblog.com, 838-232-1446).
psql:data/customer.sql:19: ERROR:  null value in column "id" of relation "customer" violates not-null constraint
DETAIL:  Failing row contains (null, Max McConnachie, Cambridge, mmcconnachie1@washingtonpost.com, 798 ...
Successfully imported postgres dump into PostgreSQL database 'unique_db'


In [81]:
!pip install pgloader

ERROR: Could not find a version that satisfies the requirement pgloader (from versions: none)
ERROR: No matching distribution found for pgloader

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
